In [19]:

!sudo apt-get update

!sudo apt-get install python3.8 -y
!sudo apt-get install python3-venv -y
!python3 -m venv invoice-env
!source invoice-env/bin/activate
!pip install numpy pandas scikit-learn spacy transformers torch onnx onnxruntime pytesseract pdf2image


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [55.4 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [927 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [51.8 kB]
Get:13 http://archive.ubuntu

**Data Collection & OCR Conversion**

In [23]:
!sudo apt-get install tesseract-ocr
!sudo apt-get install libtesseract-dev
from pdf2image import convert_from_path
import pytesseract
import os

def pdf_to_text(file_path):
    images = convert_from_path(file_path)
    text = ""
    for image in images:
        text += pytesseract.image_to_string(image, lang='eng')
    return text

# Provide the correct path to the Tesseract executable for Linux
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'
file_path = "/content/invoice_1.pdf"
text = pdf_to_text(file_path)
print(text)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libtesseract-dev is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Invoice no: 61356291

Date of issue:

Seller:

Chapman, Kim and Green
64731 James Branch
Smithmouth, NC 26872

Tax Id: 949-84-9105
IBAN: GB50ACIE59715038217063

ITEMS
No. Description Qty
1. Wine Glasses Goblets Pair Clear 5,00
Glass
2. With Hooks Stemware Storage 4,00

Multiple Uses Iron Wine Rack
Hanging Glass

3. Replacement Corkscrew Parts 1,00
Spiral Worm Wine Opener Bottle
Houdini

4. HOME ESSENTIALS GRADIENT 1,00

STEMLESS WINE GLASSES SET
OF 4 20 FL OZ (591 ml) NEW

SUMMARY

VAT [%]
10%

Total

09/06/2012

UM

each

each

each

each

Cl

 **Data Preprocessing**

In [24]:
import re

def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = re.sub(r'\n', ' ', text)   # Remove new lines
    return text

cleaned_text = preprocess_text(text)
print(cleaned_text)


Invoice no: 61356291 Date of issue: Seller: Chapman, Kim and Green 64731 James Branch Smithmouth, NC 26872 Tax Id: 949-84-9105 IBAN: GB50ACIE59715038217063 ITEMS No. Description Qty 1. Wine Glasses Goblets Pair Clear 5,00 Glass 2. With Hooks Stemware Storage 4,00 Multiple Uses Iron Wine Rack Hanging Glass 3. Replacement Corkscrew Parts 1,00 Spiral Worm Wine Opener Bottle Houdini 4. HOME ESSENTIALS GRADIENT 1,00 STEMLESS WINE GLASSES SET OF 4 20 FL OZ (591 ml) NEW SUMMARY VAT [%] 10% Total 09/06/2012 UM each each each each Client: Rodriguez-Stevens 2280 Angela Plain Hortonshire, MS 93248 Tax Id: 939-98-8477 Net price 12,00 28,08 7,50 12,99 Net worth 192,81 $ 192,81 Net worth VAT [%] 60,00 112,32 7,50 12,99 VAT 19,28 $ 19,28 10% 10% 10% 10% Gross worth 66,00 123,55 8,25 14,29 Gross worth 212,09 $ 212,09 


In [25]:
training_data = [
    (
        "Invoice no: 61356291 Date of issue: 09/06/2012 Seller: Chapman, Kim and Green 64731 James Branch Smithmouth, NC 26872 Tax Id: 949-84-9105 IBAN: GB50ACIE59715038217063 Client: Rodriguez-Stevens 2280 Angela Plain Hortonshire, MS 93248 Tax Id: 939-98-8477",
        {"entities": [(11, 19, "INVOICE_NO"), (34, 44, "DATE"), (53, 74, "SELLER"), (102, 118, "ADDRESS"), (124, 131, "TAX_ID"), (138, 161, "IBAN"), (170, 186, "CLIENT"), (189, 210, "ADDRESS"), (215, 222, "TAX_ID")]}
    )
]


**Model Training**

In [35]:
!python -m spacy download en_core_web_trf


/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 2.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [37]:
!pip install --upgrade spacy-transformers  # Upgrade spacy-transformers to the latest version
!pip install spacy-transformers


import spacy
from spacy.training import Example
from spacy.language import Language
from spacy.transformers import TransformerComponent
 # Import should work now


# Load a pre-trained model
nlp = spacy.load("en_core_web_trf")

# Add the curated transformer to the pipeline
@Language.factory("curated_transformer") # Register the component
def create_curated_transformer(nlp, name):
    return TransformerComponent(nlp.vocab)

nlp.add_pipe("curated_transformer") # Add the component to the pipeline

# Fine-tune the model
optimizer = nlp.resume_training()
for text, annotations in training_data:
    doc = nlp.make_doc(text)
    example = Example.from_dict(doc, annotations)
    nlp.update([example], sgd=optimizer)

# Save the model
nlp.to_disk("invoice_ner_model")

ModuleNotFoundError: No module named 'spacy.transformers'

**Model Optimization**

In [38]:
!pip install --upgrade spacy-transformers  # Install spacy-transformers in the same cell

import spacy
from spacy.training import Example
from spacy.language import Language
from spacy.transformers import TransformerComponent

# Load a pre-trained model
nlp = spacy.load("en_core_web_trf")

# Add the curated transformer to the pipeline
@Language.factory("curated_transformer") # Register the component
def create_curated_transformer(nlp, name):
    return TransformerComponent(nlp.vocab)

nlp.add_pipe("curated_transformer") # Add the component to the pipeline

# Fine-tune the model
optimizer = nlp.resume_training()
for text, annotations in training_data:
    doc = nlp.make_doc(text)
    example = Example.from_dict(doc, annotations)
    nlp.update([example], sgd=optimizer)

# Save the model
nlp.to_disk("invoice_ner_model")

ModuleNotFoundError: No module named 'spacy.transformers'

**Model Deployment**

In [29]:
import torch
from transformers import BertTokenizer, BertForTokenClassification

model = BertForTokenClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Dummy input for the model
dummy_input = tokenizer("This is a sample input", return_tensors="pt")

# Export the model to ONNX
torch.onnx.export(model, dummy_input['input_ids'], "bert_invoice.onnx", opset_version=11)


/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings t

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [42]:
import torch
from transformers import BertTokenizer, BertForTokenClassification

# Load the pre-trained model and tokenizer
model = BertForTokenClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Create a dummy input for the model
dummy_input = tokenizer("This is a sample input", return_tensors="pt")

# Save the model to ONNX format
onnx_model_path = "bert_invoice.onnx"
torch.onnx.export(model, dummy_input['input_ids'], onnx_model_path, opset_version=11)

# Optimize the model
import onnx
from onnxruntime.quantization import quantize_dynamic, QuantType

# Load the ONNX model
model = onnx.load(onnx_model_path)

# Quantize the model
quantized_model_path = "bert_invoice_quantized.onnx"
quantize_dynamic(onnx_model_path, quantized_model_path, weight_type=QuantType.QInt8)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [44]:
import onnxruntime as ort
from transformers import BertTokenizer

# Load the tokenizer and the model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
ort_session = ort.InferenceSession("bert_invoice_quantized.onnx")

def predict(text):
    # Add padding to match expected input size
    inputs = tokenizer(text, return_tensors="pt", padding='max_length', truncation=True, max_length=7)
    ort_inputs = {ort_session.get_inputs()[0].name: inputs['input_ids'].numpy()}
    ort_outs = ort_session.run(None, ort_inputs)
    return ort_outs

text = "Sample invoice text for testing"
prediction = predict(text)
print(prediction)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[array([[[ 0.01968104,  0.1304372 ],
        [-0.2508962 , -0.10795972],
        [-0.13319127, -0.627929  ],
        [-0.30102032, -0.24737944],
        [-0.4139374 , -0.20068727],
        [-0.21791084,  0.1058412 ],
        [-0.02148178, -0.12334016]]], dtype=float32)]
